## Final Project Submission

Please fill out:
* Student name: Jenyl Murdock
* Student pace: part time 
* Scheduled project review date/time: 
* Instructor name: Joe San Pietro
* Blog post URL:


In [ ]:
# Your code here - remember to use markdown cells for comments as well!

In [ ]:
# access the data file and get a visual of the information

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Check for multicollinearity
import seaborn as sns

# look at results for each linear regression model using statsmodels
import statsmodels.formula.api as smf

kc = pd.read_csv('kc_house_data.csv')
print (kc.shape)
kc.head(10)

Insert description of column names, grade, & condition here

For bathrooms:
Values are decimal and calculated as:
Full bathrooms (ensuite) = 1
Half bathrooms (separate )= 0.5
Powder rooms (only toilet and sink) = 0.25
(Dahlin, 2016)

In [ ]:
# check to see how each column is categorized
kc.info()

II.  Scrub the data

In [ ]:
# The id column is not needed so it can be dropped from the data
kc = kc.drop(['id'], axis=1)
kc.head()

In [ ]:
# Start with an initial data type conversion
# For columns that are obviouly categorized wrong, convert them to an appropriate data type
kc['date'] = pd.to_datetime(kc.date)
kc['condition'] = kc.condition.astype(int)
kc['grade'] = kc.grade.astype(int)

kc.info()

In [ ]:
B.  Remove null values

In [ ]:
# Check for null values in the dataset 
kc.isna().sum()

View has 63 null values.  Because this is not a significant number, these can be dropped from our data

In [ ]:
print (kc.view.unique())
print (kc.view.value_counts())

In [ ]:
# select only the data for where the view is not a nan value
# convert view to data type string so the 'nan' data can be dropped
kc['view'] = kc.view.astype(str)
kc = kc[kc.view != 'nan']  
kc['view'] = kc.view.astype(float)
kc['view'] = kc.view.astype(int)


In [ ]:
# replace null values with 0 in waterfront column and change to data type integer
kc.waterfront = kc.waterfront.fillna(value=0)
kc.waterfront = kc.waterfront.astype(int)

In [ ]:
# replace null values in yr_renovated with 0 and change to data type integer
kc.yr_renovated = kc.yr_renovated.fillna(value=0)
kc.yr_renovated = kc.yr_renovated.astype(int)

In [ ]:
# Verify that there are not any more null values
kc.isna().sum()

C.  reformat some columns and remove outliers

In [ ]:
# Convert the year built to reflect the age of the house. This format will be easier to visualize
# https://github.com/matthewsparr/King-County-Sales-Analysis/blob/master/student2.ipynb
kc['yr_built'].describe()
kc['age'] = (kc.yr_built.max() - kc.yr_built + 1)
kc = kc.drop(['yr_built'], axis=1)
kc.head(2)

The bedrooms data contains one significant outlier of 33 which could very well be a data entry error.
There are 38 houses with 7 bedrooms, 13 houses with 8 bedrooms, 6 with 9 bedrooms, 3 with 10, 1 with 11
Example code used for this information:  (kc.bedrooms == 9).sum()

In [ ]:
# Use only the values from 1-6 bedrooms
print (kc.bedrooms.value_counts())
kc = kc[kc['bedrooms'] < 7]
kc.bedrooms.unique()

In [ ]:
# Check columns for unusable data by looking at the unique values for each variable

In [ ]:
kc.floors.unique()

In [ ]:
kc.bathrooms.unique()
kc.condition.unique()
kc.grade.unique()
kc.sqft_living.unique() 
kc.sqft_lot.unique()
kc.condition.unique()
kc.grade.unique()
kc.sqft_above.unique()
kc.sqft_basement.unique()  # '?' in the data

(kc.sqft_basement =='?').sum()  # there are 450 '?' in this column
#(kc.sqft_basement == '0.0').sum()  # there are 12798 '0.0' values in this column

In [ ]:
# Replace the '?' values with 0 i the sqft_basement data
kc['sqft_basement'] = (kc['sqft_basement'].replace('?', 0)).astype(float)
kc['sqft_basement'] = kc['sqft_basement'].astype(int)

# Instead of square feet values for the basement, let's just use a categorical value of 1 for if it has a basement
# and 0 for if it does not have a basement.
# https://github.com/matthewsparr/King-County-Sales-Analysis/blob/master/student2.ipynb
kc['basement'] = kc['sqft_basement'].apply(lambda row: 0 if row == 0 else 1) 
print(kc['basement'].unique())

In [ ]:
print (kc['view'].value_counts())

# A significant majority of the values for the variable view are 0, let's just use a categorical value of 1 for 
# if it has a view and 0 for if it does not have a view.
# https://github.com/matthewsparr/King-County-Sales-Analysis/blob/master/student2.ipynb

kc['has_view'] = kc['view'].apply(lambda row: 1 if row != 0 else 0)
print (kc['has_view'].unique())

In [ ]:
# Check that all data is converted to appropriate data type
kc.info()

In [ ]:
# Set up the matplotlib figure and generate a heatmap of the data
f, ax = plt.subplots(figsize=(10, 8))

sns.heatmap(kc.corr(), center = 0, linewidths = .25);

It looks like sqft_living, grade has high collinearity with some other features

In [ ]:
# Get an initial visualization of the data
# Note:  extra white space on either side of the graph indicates outliers which need to be removed
kc.hist(figsize=(15,15));

In [ ]:
# Normalize the data using log transformation
#   the variables basement, age, has_view, yr_renovated cannot be log transformed due to their values of 0
import numpy as np
kc_log = pd.DataFrame([])

kc_log['bathrooms'] = np.log(kc.bathrooms)
kc_log['bedrooms'] = np.log(kc.bedrooms)
kc_log['condition'] = np.log(kc.condition)
kc_log['floors'] = np.log(kc.floors)
kc_log['grade'] = np.log(kc.grade)
kc_log['price'] = np.log(kc.price)
kc_log['sqft_above'] = np.log(kc.sqft_above)
kc_log['sqft_living'] = np.log(kc.sqft_living)
kc_log['sqft_lot'] = np.log(kc.sqft_lot)
kc_log['sqft_lot15'] = np.log(kc.sqft_lot15)
kc_log['zipcode'] = np.log(kc.zipcode)

# look at the histograms of the transformed data
kc_log.hist(figsize = (15,15));

In [ ]:
# Convert categorical columns to numeric format with one-hot encoding

#  Change the selected categorical variables to data type'category'
#kc.condition = kc.condition.astype('category')
#kc.floors = kc.floors.astype('category')
#kc.grade = kc.grade.astype('category')
kc.has_view = kc.has_view.astype('category')
kc.waterfront = kc.waterfront.astype('category')
kc.basement = kc.basement.astype('category')

#cond_dummies = pd.get_dummies(kc.condition, prefix = 'condition')
#floors_dummies = pd.get_dummies(kc.floors, prefix = 'floors')
#grade_dummies = pd.get_dummies(kc.grade, prefix = 'grade')
has_view_dummies = pd.get_dummies(kc.has_view, prefix = 'view')
water_dummies = pd.get_dummies(kc.waterfront, prefix = 'waterfront')
basement_dummies = pd.get_dummies(kc.basement, prefix = 'basement')



kc = kc.drop(['view', 'has_view', 'waterfront', 'sqft_basement', 'basement'], axis=1)
kc = pd.concat([kc, has_view_dummies, water_dummies, basement_dummies], axis=1)


In [ ]:
# set the max_columns attribute in order to view all of the column names with the categorical data
pd.set_option('display.max_columns', None)
kc.head()

In [ ]:
# run an OLS regression using the log transformed data
col_names = kc_log.columns.drop(['price'])
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p_value']]

for idx, val in enumerate(col_names):
    print ('Kings County:  Price ~ ' + val)
    print ('-------------------------------------')
    
    f = 'price~' + val
    model = smf.ols(formula = f, data = kc_log).fit()
    X_new = pd.DataFrame({val: [kc_log[val].min(), kc_log[val].max()]});
    predic = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print (results[idx+1])

In [ ]:
# Convert the results table to a dataframe
print ('Regression Model with log transformed data')
pd.DataFrame(results)

The p-value does not give us any significant value that shows a variable should be removed.
The three highest r-squared values are for grade, sqft_above, and sqft_living 

In [ ]:
# check the column indices for reference locations
kc.head(2)

In [ ]:
# run an initial OLS regression between the continuous independent variables and the dependent variable
# do not include id, date and the categorical data, so use columns 2 through 15
col_names  = kc.columns[2:17]
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p_value']]

for idx, val in enumerate(col_names):
    print()
    print ('Kings County:  Price ~ ' + val)
    print ('-------------------------------------')
    
    f = 'price~' + val
    model = smf.ols(formula = f, data = kc).fit()
    X_new = pd.DataFrame({val: [kc[val].min(), kc[val].max()]});
    predic = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print (results[idx+1])

In [ ]:
# Convert the continuous variables results table to a dataframe
print ()
print ('OLS with continuous variables - 1st Iteration')
pd.DataFrame(results)

Remove obvious variables such as:
    p-value > .05
    r squared value that are very close to 0

In [ ]:
# In this situation, longitude and latitude are not very predictive and can be eliminated.  They need each other 
# in order to have any meaning and for now, zipcode does a similar function in one variable.

# The variable sqft_living has a lot of correlation as seen in the heat map.  It also has the highest r-squared
# value of our continueous variables.

drop_cols = ['lat', 'long', 'sqft_living']
kc = kc.drop(drop_cols, axis=1)


After removing the first round of variables, run the OLS again on the remaining continuous variables.
Since a few columns have been deleted, the column indices need to be adjusted

In [ ]:
# check the column indices as some have been removed
kc.head(2)

In [ ]:
col_names = kc.columns[2:14]

results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p_value']]

for idx, val in enumerate(col_names):
    print()
    print ('Kings County:  Price ~ ' + val)
    print ('-------------------------------------')
    
    f = 'price~' + val
    model = smf.ols(formula = f, data = kc).fit()
    X_new = pd.DataFrame({val: [kc[val].min(), kc[val].max()]});
    predic = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print (results[idx+1])

In [ ]:
# Convert the continuous variables results table to a dataframe
print ()
print ('OLS with continuous variables - 2nd Iteration')
pd.DataFrame(results)

In [ ]:
# drop columns grade amd sqft_living15
drop_cols =['grade','sqft_living15']
kc = kc.drop(drop_cols, axis=1)


In [ ]:
kc.head(10)

In [ ]:
# now run an initial OLS regression with the categorial data (columns 12 and on) and the dependent variable
col_names = kc.columns[12:]  # check the column starting point
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p_value']]

for idx, val in enumerate(col_names):
    print ('Kings County:  Price ~ ' + val)
    print ('-------------------------------------')
    
    f = 'price~' + val
    model = smf.ols(formula = f, data = kc).fit()
    X_new = pd.DataFrame({val: [kc[val].min(), kc[val].max()]});
    predic = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print (results[idx+1])

In [ ]:
 # convert the categorical variables results table to a dataframe
print ()
print ('OLS with categorical variables')
pd.DataFrame(results)

In [ ]:
# Evaluate whether any categorical variables should be eliminated

In [ ]:
# not sure about which ones.  view has higher than waterfront or basement, so maybe view goes
drop_cols = ['view_0', 'view_1']
kc = kc.drop(drop_cols, axis=1)

A correlation is consider high if its absolute value is around 0.7 - 0.8.
Let's use 0.75 as a guide to see how many high correlations there are.
There were no variables that have that high of correlation.  Let's try 0.7.
This showed a high correlation between sqft_lot and sqft_lot15.  Let's go ahead and
drop the sqft_lot15 variable

In [ ]:
abs(kc.corr() > 0.7)
kc = kc.drop(['sqft_lot15'], axis=1)

In [ ]:
kc.head(2)

In [ ]:
# now run an OLS regression with the remaining data (columns 2 and on) and the dependent variable
col_names = kc.columns[2:]  # adjust the column starting point
results = [['ind_var', 'r_squared', 'intercept', 'slope', 'p_value']]

for idx, val in enumerate(col_names):
    print ('Kings County:  Price ~ ' + val)
    print ('-------------------------------------')
    
    f = 'price~' + val
    model = smf.ols(formula = f, data = kc).fit()
    X_new = pd.DataFrame({val: [kc[val].min(), kc[val].max()]});
    predic = model.predict(X_new)
    results.append([val, model.rsquared, model.params[0], model.params[1], model.pvalues[1]])
    print (results[idx+1])

In [ ]:
 # convert the remaining variables results table to a dataframe
print ()
print ('OLS with remaining variables')
pd.DataFrame(results)

Personal opinion, I sthink that yr_renovated should be removed and possibly basement
Date can also be dropped

In [ ]:
drop_cols = ['date', 'yr_renovated', 'basement_0', 'basement_1']
kc = kc.drop(drop_cols, axis=1)

Model Validation - using a train-test-split

In [ ]:
X = kc.drop(['price'], axis=1)
y = kc['price']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2)

from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(X_train, y_train)

yhat_train = linreg.predict(X_train)
yhat_test = linreg.predict(X_test)

# look at the residuals
train_residuals = yhat_train - y_train
test_residuals = yhat_test - y_test

print ('Training Data Residuals: ', train_residuals)
print ()
print ()
print ('Test Data Residuals: ', test_residuals)




In [ ]:
# mean squarred error
mse_train = np.sum((y_train - yhat_train)**2) / len(y_train)
mse_test= np.sum((y_test - yhat_test)**2) / len(y_test)

print ('Train Mean Squarred Error: ', mse_train)
print ('Test Mean Squarred Error: ', mse_test)

In [ ]:
# I don't think the mse's look good :(